In [2]:
#r "nuget: FSharp.Stats, 0.5.1-preview.1"
#r "nuget: Plotly.NET.Interactive, 4.2.1"
#r "nuget: Cytoscape.NET.Interactive, 0.2.0"


open FSharp.Stats
open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.LayoutObjects
open FSharp.Data
open Cytoscape.NET
open System

// use a script to import data in all notebooks > 1
#load "import.fsx"
open Import
let orders = Import.orders

Installed Packages Cytoscape.NET.Interactive, 0.2.0 FSharp.Stats, 0.5.1-preview.1 Plotly.NET.Interactive, 4.2.1

In [4]:
open FSharp.Stats.Signal

let rawData =
    orders
    |> Array.filter (fun x -> x.Name = "Benedikt V.") //
    |> Array.groupBy (fun x -> x.DateTime.Date)
    |> Array.map (fun (date,x) -> date,float x.Length)

let paddedData =
    rawData
    |> fun x -> 
        Padding.pad x 
            1440 
            30000 
            Padding.HelperFunctions.Time.getDiffMinutes 
            Padding.HelperFunctions.Time.addToXValueMinutes 
            1000
            Padding.BorderPaddingMethod.Random 
            Padding.InternalPaddingMethod.Zero
            Padding.HugeGapPaddingMethod.Zero

let rickerArray = 
    [|100;120;150;180;230;290;360;440;530;630;740;|] |> Array.map (fun x -> Wavelet.createRicker (x**1.8))


let transformedData = 
    rickerArray
    |> Array.map (fun wavelet -> ContinuousWavelet.transform paddedData Padding.HelperFunctions.Time.getDiffMinutes 1000 wavelet)

    
///combining the raw and transformed data in one chart
let combinedChart =
    //CWT-chart
    let heatmap =
        let rowNames,colNames = 
            transformedData.[0] |> Array.mapi (fun i (x,_) -> string i, string x) |> Array.unzip
        transformedData
        |> JaggedArray.map snd
        |> fun x -> Chart.Heatmap(x,colNames=colNames,rowNames=rowNames,ShowScale=false)
        |> Chart.withAxisAnchor(X=1)
        |> Chart.withAxisAnchor(Y=1)

    //Rawchart
    let rawChart = 
        //Chart.Column (rawData,LineColor = Color.fromHex "#1f77b4",Name = "rawData")
        Chart.Column (rawData,MarkerColor = Color.fromHex "#1f77b4",Name = "rawData")
        |> Chart.withAxisAnchor(X=2)
        |> Chart.withAxisAnchor(Y=2) 

    //combine the charts and add additional styling
    Chart.combine([heatmap;rawChart])
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle("Time",Side=Side.Bottom,Id=SubPlotId.XAxis 2,ShowGrid=false)
    //|> Chart.withXAxisStyle("Time",ShowGrid=false)
    |> Chart.withXAxisStyle("", Side=Side.Top,ShowGrid=false, Id=SubPlotId.XAxis 1,Overlaying=LinearAxisId.X 2)
    |> Chart.withYAxisStyle("drink count", MinMax=(-25.,30.), Side=Side.Left,Id=SubPlotId.YAxis 2)
    |> Chart.withYAxisStyle(
        "Rickerscales", MinMax=(0.,32.),ShowGrid=false, Side=Side.Right,
        Id=SubPlotId.YAxis 1,Overlaying=LinearAxisId.Y 2)
    |> Chart.withLegend true

combinedChart

<!-- Plotly chart will be drawn inside this DIV -->